In [3]:
!pip install selenium

  Using cached selenium-4.35.0-py3-none-any.whl.metadata (7.4 kB)
  Using cached urllib3-2.5.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached trio-0.30.0-py3-none-any.whl.metadata (8.5 kB)
  Using cached trio_websocket-0.12.2-py3-none-any.whl.metadata (5.1 kB)
  Using cached certifi-2025.8.3-py3-none-any.whl.metadata (2.4 kB)
  Using cached typing_extensions-4.14.1-py3-none-any.whl.metadata (3.0 kB)
  Using cached attrs-25.3.0-py3-none-any.whl.metadata (10 kB)
  Using cached outcome-1.3.0.post0-py2.py3-none-any.whl.metadata (2.6 kB)
  Using cached wsproto-1.2.0-py3-none-any.whl.metadata (5.6 kB)
   ---------------------------------------- 0.0/9.6 MB ? eta -:--:--
   --------- ------------------------------ 2.4/9.6 MB 13.4 MB/s eta 0:00:01
   ------------------ --------------------- 4.5/9.6 MB 11.7 MB/s eta 0:00:01
   ---------------------------- ----------- 6.8/9.6 MB 11.7 MB/s eta 0:00:01
   -------------------------------- ------- 7.9/9.6 MB 10.6 MB/s eta 0:00:01
   --------------

In [5]:
!pip install webdriver-manager

  Using cached webdriver_manager-4.0.2-py2.py3-none-any.whl.metadata (12 kB)


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time

def get_imdb_top250():
    imdb_url = "https://www.imdb.com/chart/top/"
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
    driver.get(imdb_url)
    time.sleep(3)  # wait for page to load

    movie_list = []
    movie_items = driver.find_elements(By.CSS_SELECTOR, "ul.ipc-metadata-list li")

    for rank, item in enumerate(movie_items, start=1):
        try:
            movie_title = item.find_element(By.CSS_SELECTOR, "h3").text
            release_year = item.find_element(By.CSS_SELECTOR, "span.cli-title-metadata-item").text
            imdb_score = item.find_element(By.CSS_SELECTOR, "span.ipc-rating-star--rating").text
            movie_list.append([rank, movie_title, release_year, imdb_score])
        except:
            continue

    driver.quit()
    df = pd.DataFrame(movie_list, columns=["Rank", "Movie Title", "Year of Release", "IMDB Rating"])
    df.to_csv("imdb_top250.csv", index=False)
    print("IMDB Top 250 scraping finished! Data saved to imdb_top250.csv")

get_imdb_top250()


IMDB Top 250 scraping finished! Data saved to imdb_top250.csv


In [11]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
BASE_URL = "https://books.toscrape.com/catalogue/page-{}.html"
titles = []
prices = []
availabilities = []
ratings = []
rating_map = {
    "One": "1",
    "Two": "2",
    "Three": "3",
    "Four": "4",
    "Five": "5"
}
for page in range(1, 51):
    url = BASE_URL.format(page)
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Failed to retrieve page {page}")
        continue
    soup = BeautifulSoup(response.text, "html.parser")
    books = soup.find_all("article", class_="product_pod")
    for book in books:
        title = book.h3.a["title"]
        price = book.find("p", class_="price_color").text.strip()
        availability = book.find("p", class_="instock availability").text.strip()
        star_class = book.p["class"][1]  
        rating = rating_map.get(star_class, "Not Rated")
        titles.append(title)
        prices.append(price)
        availabilities.append(availability)
        ratings.append(rating)
df = pd.DataFrame({
    "Title": titles,
    "Price": prices,
    "Availability": availabilities,
    "Star Rating": ratings
})
df.to_csv("books.csv", index=False, encoding="utf-8")
print("Scraping completed! Data saved to books.csv")


Scraping completed! Data saved to books.csv


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

base_url = "https://www.timeanddate.com"
main_url = base_url + "/weather/"

response = requests.get(main_url)
soup = BeautifulSoup(response.text, "html.parser")
table = soup.find("table", class_="zebra")

cities, temps, conditions = [], [], []

for row in table.find_all("tr")[1:]:
    cols = row.find_all("td")
    if len(cols) >= 1:
        city_name = cols[0].get_text(strip=True)
        city_link_tag = cols[0].find("a")
        if city_link_tag:
            city_url = base_url + city_link_tag['href']
            city_resp = requests.get(city_url)
            city_soup = BeautifulSoup(city_resp.text, "html.parser")
            
            # Extract temperature
            temp_tag = city_soup.find("div", class_="h2")
            temp = temp_tag.get_text(strip=True) if temp_tag else "N/A"
            
            # Extract weather condition
            cond_tag = city_soup.find("div", class_="h2").find_next_sibling("p")
            condition = cond_tag.get_text(strip=True) if cond_tag else "N/A"
            
            cities.append(city_name)
            temps.append(temp)
            conditions.append(condition)

df = pd.DataFrame({"City": cities, "Temperature": temps, "Condition": conditions})
df.to_csv("weather.csv", index=False)
print(df.head())


          City Temperature        Condition
0        Accra       25 °C   Mostly cloudy.
1  Addis Ababa       16 °C   Partly cloudy.
2     Adelaide       13 °C      Quite cool.
3      Algiers       28 °C  Passing clouds.
4       Almaty       17 °C   Partly cloudy.
